## Create your own API key
- Call the register function once to create your API key and store it for future use.
- By default, your API key will be stored in a file named api_key.txt in the same folder.
- You need to pass this API key in the request header when calling the Judge LLM (next cell). 
```
headers = {
    "Content-Type": "application/json",
    "X-API-Key": "your_api_gey"
}
```
- *You will only run the following code once*

In [1]:
import requests
import json
import logging

logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)


# Function to register and get API key
def register_user():
    register_url = "http://127.0.0.1:8899/v1/register"
    try:
        response = requests.post(register_url)
        response.raise_for_status()
        api_key = response.json()["api_key"]
        logger.info("Successfully registered and received API key")
        return api_key
    except requests.exceptions.RequestException as e:
        logger.error(f"Failed to register user: {str(e)}")
        raise

# Get or create API key
try:
    api_key_file = "api_key.txt"
    with open(api_key_file, "r") as f:
        api_key = f.read().strip()
    logger.info("Loaded existing API key: {0} from the file: {1}".format(api_key, api_key_file))
except FileNotFoundError:
    api_key = register_user()
    with open(api_key_file, "w") as f:
        f.write(api_key)
    logger.info("Your API is {0}; it is saved in the file: {1} ".format(api_key, api_key_file))

2024-10-17 17:33:19,607 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): 127.0.0.1:8899
2024-10-17 17:33:19,612 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8899 "POST /v1/register HTTP/11" 200 50
2024-10-17 17:33:19,654 - __main__ - INFO - Successfully registered and received API key
2024-10-17 17:33:19,656 - __main__ - INFO - Your API is e2d75c40-9e0e-4182-8700-a7766fd18633; it is saved in the file: api_key.txt 


## API reference for the Judge
### Input & Output Format:
- **Input:** should be the natural language question and the model-generated FOL statement separated by a = sign.
- **Output:** would be ROUGE F1 score for the model-generated response.
    - Example: ROUGE-L F1: 0.57

**N.B.** ROUGE score is the similarity between a machine-generated summary or translation and a human-created reference

In [2]:
import requests
import json

# call this function to validate the fol_question generated by the fine-tuned model
def call_judge(API_key, natural_language_question, fol_question):
    api_url = "http://127.0.0.1:8899/v1/judge"
    headers = {
        "Content-Type": "application/json",
        "X-API-Key": API_key #"your_api_gey"
    }

    params = {    
        "max_new_tokens": 512,
        "temperature": 0.7,
        "do_sample":True,
        "temperature":0.6,
        "top_p":0.9,
        "prompt":"{0}?={1}".format(natural_language_question, fol_question),    
    }

    response = requests.post(api_url, headers=headers, data=json.dumps(params))
    response.raise_for_status()
    result = response.json()["choices"][0]["text"].strip()
    return result
# end Judge

In [3]:
# Sample call
output = call_judge(API_key="b1159222-167d-468f-b724-2f23f0525c10", # Change this API key to yours
           natural_language_question="Does the black SUV on the left move at a constant speed",
           fol_question="TypeOf(x, SUV)^ColorOf(x, Black)^ConstantSpeed(x)^TypeOf(y, Car)"
           )

print(output)

2024-10-17 17:33:25,707 - urllib3.connectionpool - DEBUG - Starting new HTTP connection (1): 127.0.0.1:8899
2024-10-17 17:33:27,799 - urllib3.connectionpool - DEBUG - http://127.0.0.1:8899 "POST /v1/judge HTTP/11" 200 212


ROUGE-L F1: 0.75


## Interpretation of ROUGE-L F1 Scores for FOL Statements

ROUGE-L (Longest Common Subsequence) F1 scores are used to evaluate the quality of generated First-Order Logic (FOL) statements. Here's a guide to interpreting these scores:

### Score Range Interpretation

| Score Range | Interpretation                    | Action                                      |
|-------------|-----------------------------------|---------------------------------------------|
| 0.0 - 0.4   | Poor to Unacceptable              | Needs significant improvement               |
| 0.4 - 0.7   | Acceptable, but room for improvement | Review and refine the FOL statement      |
| 0.7 - 1.0   | Good to Excellent                 | Meets or exceeds expectations               |

### Key Points

1. **Minimum Acceptable Score**: A score above 0.4 is considered acceptable, but there's still room for improvement.
2. **Target Score**: Aim for scores above 0.7 for high-quality FOL statements.
3. **Perfect Score**: A score of 1.0 indicates a perfect match, which is rare in practice.